In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import talib as ta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from xgboost import XGBClassifier
import talib.abstract as ta
import pandas_ta as pta
import freqtrade.vendor.qtpylib.indicators as qtpylib

df1 = pd.read_json('2022-10-05 07:48:23.436271.json')                      
df2 = pd.read_json('2022-10-05 10:33:39.353703.json') 

In [10]:
df1['long_profit'] = (df1['profit_ratio'].multiply(100)).round(2)
df1['short_profit'] = (df2['profit_ratio'].multiply(100)).round(2)
# df = df1[df.long_profit.notna()]
# df[df.long_profit.notna()]
df = df1
# df = df[-10000:]
 
df.loc[(df.long_profit > 0) & (df.short_profit < 0), 'label'] = 'long'
df.loc[(df.long_profit < 0) & (df.short_profit > 0), 'label'] = 'short'
df.loc[(~df['label'].isin(['long', 'short'])) & (df.long_profit.notna()), 'label'] = 'hold'
""" prepare labels """
df.loc[(df['trade'] == 1)& (df['macd'] > df['macdsignal']), 'position'] = 'bottom' 
df.loc[(df['trade'] == 1)& (df['macd'] < df['macdsignal']), 'position'] = 'top'
df.loc[(df['trade'] == 1)& (df['macd'] > df['macdsignal']), 'position'] = 'long' 
df.loc[(df['trade'] == 1)& (df['macd'] < df['macdsignal']), 'position'] = 'short'
df[df.long_profit.notna()]




,date,open,high,low,close,volume,macd,macdsignal,macdhist,cci,enter_long,trade,is_short,profit_ratio,label,long_profit,short_profit,position
101,2019-11-27 12:48:00,148.25,148.25,147.75,148.11,359.330,1.039415,1.294305,-0.254890,39.572072,1.0,1.0,0.0,-0.004378,short,-0.44,0.45,short
102,2019-11-27 12:51:00,147.94,148.07,147.51,147.98,377.030,0.976753,1.230795,-0.254041,-8.907642,1.0,1.0,0.0,-0.003508,short,-0.35,0.24,short
103,2019-11-27 12:54:00,147.97,147.97,147.40,147.59,452.110,0.885417,1.161719,-0.276302,-75.425504,1.0,1.0,0.0,-0.004386,short,-0.44,0.26,short
104,2019-11-27 12:57:00,147.59,147.63,146.94,147.38,208.430,0.787015,1.086778,-0.299763,-160.250975,1.0,1.0,0.0,-0.004396,hold,-0.44,-0.07,short
105,2019-11-27 13:00:00,147.38,147.83,147.26,147.65,378.020,0.722489,1.013920,-0.291432,-73.864933,1.0,1.0,0.0,0.001431,long,0.14,-0.36,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499701,2022-10-03 08:48:00,1291.04,1291.98,1290.30,1291.40,3492.861,0.077094,-0.151638,0.228732,74.670879,1.0,1.0,0.0,-0.002018,short,-0.20,0.12,long
499702,2022-10-03 08:51:00,1291.40,1291.86,1290.54,1291.71,2916.172,0.147993,-0.091712,0.239705,74.332765,1.0,1.0,0.0,-0.002304,short,-0.23,0.15,long
499703,2022-10-03 08:54:00,1291.72,1291.87,1290.73,1290.95,4836.515,0.141228,-0.045124,0.186352,53.451299,1.0,1.0,0.0,-0.002543,short,-0.25,0.17,long
499704,2022-10-03 08:57:00,1290.95,1291.09,1290.22,1290.82,3393.288,0.123948,-0.011309,0.135258,6.859768,1.0,1.0,0.0,-0.001949,short,-0.19,0.11,long


In [17]:
df.label = df.label.shift(-1)

In [25]:
df.profit_ratio = df.profit_ratio.shift(-1)

In [34]:
df = df1.copy()

In [39]:
df[:20]

,date,open,high,low,close,volume,macd,macdsignal,macdhist,cci,enter_long,trade,is_short,profit_ratio,label,long_profit,short_profit
0,2019-11-27 07:45:00,146.00,146.00,146.00,146.00,0.01,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,None,NaN,NaN
1,2019-11-27 07:48:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,2019-11-27 07:51:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,2019-11-27 07:54:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,2019-11-27 07:57:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
5,2019-11-27 08:00:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
6,2019-11-27 08:03:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
7,2019-11-27 08:06:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
8,2019-11-27 08:09:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
9,2019-11-27 08:12:00,146.00,146.00,146.00,146.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
